In [2]:
!pip install tpot

In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from tpot import TPOTRegressor

SEED = 12345
N_SPLIT = 3

In [4]:
from google.colab import drive
drive.mount("/content/drive/")
INPUT_PATH_1 = "/content/drive/My Drive/HousePrices/input/"
INPUT_PATH_2 = "/content/drive/My Drive/HousePrices/features/"
OUTPUT_PATH = "/content/drive/My Drive/HousePrices/prediction/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
df = pd.read_csv(INPUT_PATH_2 + "data_v0.csv")
print(df.shape)
df.head(3)

(2919, 82)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train_test
0,784,85,3.0,NaN,9101,1.0,NaN,0.0,3.0,0.0,...,NaN,NaN,NaN,0,7,2009,8.0,4.0,165500.0,1
1,512,120,3.0,40.0,6792,1.0,NaN,0.0,3.0,0.0,...,NaN,NaN,NaN,0,3,2006,6.0,5.0,202665.0,1
2,1072,60,3.0,78.0,11700,1.0,NaN,3.0,3.0,0.0,...,NaN,NaN,NaN,0,5,2009,8.0,4.0,154000.0,1


In [6]:
df_train = df.loc[(df['train_test'].isin([1,2])), :].drop(columns=['train_test','Id'], axis=1)
df_train.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,85,3.0,NaN,9101,1.0,NaN,0.0,3.0,0.0,0.0,...,0,NaN,NaN,NaN,0,7,2009,8.0,4.0,165500.0
1,120,3.0,40.0,6792,1.0,NaN,0.0,3.0,0.0,4.0,...,0,NaN,NaN,NaN,0,3,2006,6.0,5.0,202665.0
2,60,3.0,78.0,11700,1.0,NaN,3.0,3.0,0.0,4.0,...,0,NaN,NaN,NaN,0,5,2009,8.0,4.0,154000.0


In [7]:
model = TPOTRegressor(scoring='neg_mean_squared_error', generations=5, population_size=5, cv=N_SPLIT, random_state=SEED, verbosity=2, n_jobs=-1)
model.fit(df_train.drop(columns=['SalePrice'], axis=1), df_train['SalePrice'])

Imputing missing values in feature set


Optimization Progress:   0%|          | 0/30 [00:00<?, ?pipeline/s]


Best pipeline: LassoLarsCV(input_matrix, normalize=False)


TPOTRegressor(cv=3, generations=5, n_jobs=-1, population_size=5,
              random_state=12345, scoring='neg_mean_squared_error',
              verbosity=2)

In [10]:
model.fitted_pipeline_

Pipeline(steps=[('lassolarscv', LassoLarsCV(normalize=False))])

In [11]:
# submission
df_sub = df.loc[(df['train_test']==3), :].drop(columns=['train_test','Id','SalePrice'], axis=1)
print(df_sub.shape)
df_sub.head(3)

(1459, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1460,20,2.0,80.0,11622,1.0,NaN,3.0,3.0,0.0,4.0,...,120,0,NaN,2.0,NaN,0,6,2010,8.0,4.0
1461,20,3.0,81.0,14267,1.0,NaN,0.0,3.0,0.0,0.0,...,0,0,NaN,NaN,0.0,12500,6,2010,8.0,4.0
1462,60,3.0,74.0,13830,1.0,NaN,0.0,3.0,0.0,4.0,...,0,0,NaN,2.0,NaN,0,3,2010,8.0,4.0


In [12]:
df_out = df.loc[(df['train_test']==3), :][['Id']]
df_out.head(3)

,Id
1460,1461
1461,1462
1462,1463


In [13]:
# 予測
df_out['SalePrice'] = model.predict(df_sub)
print(df_out['SalePrice'].mean())

Imputing missing values in feature set
179091.7728477914


In [15]:
df_out.to_csv(OUTPUT_PATH + "sub_tpot_v0.csv", index=False)
df_out.head(3)

,Id,SalePrice
1460,1461,109171.680778
1461,1462,157280.183982
1462,1463,167943.821100
